In [10]:
import pprint
import requests
import sys

In [30]:
api_key = "9b656c6f5237727cdf1e35288a7afa8c"

loc = [
  'Buenos Aires, Argentina',
  'Guangzhou, China',
  'Wichita, Kansas',
  'Niskayuna, New York',
  'Gwangmyeong, South Korea',
  'Taipei, Taiwan',
  'Nanaimo, British Columbia',
  'Chennai, India',
  'Barrington, Illinois',
  'Littleton, Colorado',
  'Peterhead, Scotland',
  'Vizag, India',
  'Des Moines, Iowa',
  'Beijing, China',
  'Killeen, Texas',
  'Morehead City, North Carolina'
]

city = loc[-1]

Morehead City, North Carolina


In [31]:
geo_URL = 'http://api.openweathermap.org/geo/1.0/direct'

### add below loop when finished 
### for city in loc:
geo = f'{geo_URL}?q={city}&limit=5&appid={api_key}'
resp = requests.get( geo )

if resp.status_code != 200: # Failure?
    print( f'Error geocoding {city}: {resp.status_code}' )
    sys.exit( 1 )

# OpenWeatherMap returns a list of matching cities, up to the limit specified
# in the API call; even if you only ask for one city (limit=5), it's still
# returned as a 1-element list

if len( resp.json() ) == 0: # No such city?
    print( f'Error locating city {city}; {resp.status_code}' )
    sys.exit( 2 )

json = resp.json()
if type( json ) == list: # List of cities?
    lat = json[ 0 ][ 'lat' ]
    lon = json[ 0 ][ 'lon' ]
else: # Unknown city?
    print( f'Error, invalid data returned for city {city}, {resp.status_code}' )
    sys.exit( 3 )

# Use Peterhead's latitude and longitude to get its 5-day forecast in 3-hour
# blocks

forecast_URL = 'http://api.openweathermap.org/data/2.5/forecast'
forecast = f'{forecast_URL}?lat={lat}&lon={lon}&appid={api_key}'
resp = requests.get( forecast )

if resp.status_code != 200: # Failure?
    print( f'Error retrieving data: {resp.status_code}' )
    sys.exit( 4 )

# Pretty-print the resulting JSON forecast for the first 3 hour block

print( f'{city}:' )
data = resp.json()
printer = pprint.PrettyPrinter( width=80, compact=True )
printer.pprint( data[ 'list' ][ 0 ] )

Morehead City, North Carolina:
{'clouds': {'all': 16},
 'dt': 1720332000,
 'dt_txt': '2024-07-07 06:00:00',
 'main': {'feels_like': 303.84,
          'grnd_level': 1015,
          'humidity': 87,
          'pressure': 1015,
          'sea_level': 1015,
          'temp': 300.25,
          'temp_kf': 0.59,
          'temp_max': 300.25,
          'temp_min': 299.66},
 'pop': 0.84,
 'sys': {'pod': 'n'},
 'visibility': 10000,
 'weather': [{'description': 'few clouds',
              'icon': '02n',
              'id': 801,
              'main': 'Clouds'}],
 'wind': {'deg': 244, 'gust': 9.26, 'speed': 5.88}}


In [34]:
forecast_data = data[ 'list' ][ 0 ]
temp_max = forecast_data['main']['temp_max']
temp_min = forecast_data['main']['temp_min']
print(temp_max)

299.79


In [38]:
for i in range(24):
    forecast_data = data[ 'list' ][ i ]
    print(forecast_data['dt_txt'])

2024-07-07 06:00:00
2024-07-07 09:00:00
2024-07-07 12:00:00
2024-07-07 15:00:00
2024-07-07 18:00:00
2024-07-07 21:00:00
2024-07-08 00:00:00
2024-07-08 03:00:00
2024-07-08 06:00:00
2024-07-08 09:00:00
2024-07-08 12:00:00
2024-07-08 15:00:00
2024-07-08 18:00:00
2024-07-08 21:00:00
2024-07-09 00:00:00
2024-07-09 03:00:00
2024-07-09 06:00:00
2024-07-09 09:00:00
2024-07-09 12:00:00
2024-07-09 15:00:00
2024-07-09 18:00:00
2024-07-09 21:00:00
2024-07-10 00:00:00
2024-07-10 03:00:00
